In [ ]:
import rasterio

# Path to the GeoTIFF file
file_path = "/data3/interns/NRT_CO2_Emission_Map_Project/PinyiLu_work/odiac/tif/2019/odiac2024_1km_excl_intl_1901.tif"

# Open the file and print metadata and array details
with rasterio.open(file_path) as src:
    print("Driver:", src.driver)
    print("CRS:", src.crs)
    print("Width, Height:", src.width, src.height)
    print("Number of bands:", src.count)
    print("Data type of each band:", src.dtypes)
    print("Affine transform:", src.transform)
    
    # Read the first band to inspect array shape and dtype
    data = src.read(1)
    print("Array shape (rows, cols):", data.shape)
    print("Array dtype:", data.dtype)
    print("Value range: min =", data.min(), ", max =", data.max())


In [ ]:
import rasterio

file_path = "/data3/interns/NRT_CO2_Emission_Map_Project/PinyiLu_work/odiac/tif/2019/odiac2024_1km_excl_intl_1901.tif"

with rasterio.open(file_path) as src:
    print("Global tags:", src.tags())
    # Print tags for relevant namespaces
    namespaces = src.tags(ns=None)
    print("\nAll namespaces and their tags:")
    for ns in src.tags().keys():
        print(f"Namespace '{ns}':", src.tags(ns=ns))
    # TIFF-specific tags
    try:
        print("\nTIFF tags:", src.tags(ns='TIFF'))
    except Exception as e:
        print("\nTIFF tags not available:", e)


In [ ]:
import importlib.metadata as _md

_orig_distributions = _md.distributions

def _safe_distributions(*args, **kwargs):
    for dist in _orig_distributions(*args, **kwargs):
        if getattr(dist, "metadata", None) is not None:
            yield dist

_md.distributions = _safe_distributions

import os
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr 
from tqdm import tqdm

template_nc = "/data3/interns/NRT_CO2_Emission_Map_Project/HaoHu_work/XCO2_resample/global_grid_0.1_2019_2025.nc"
tif_root    = "/data3/interns/NRT_CO2_Emission_Map_Project/PinyiLu_work/odiac/tif"
output_nc   = "odiac_interp_2019_2025.nc"

ds = xr.open_dataset(template_nc)
lat = ds["lat"]
lon = ds["lon"]
times = ds["time"].values

n_t, n_y, n_x = len(times), len(lat), len(lon)
data = np.full((n_t, n_y, n_x), np.nan, dtype=np.float32)

prev_month = None
monthly_interp = None
for it, t in enumerate(tqdm(times, total=n_t, desc="date")):
    dt = pd.to_datetime(t)
    year, month = dt.year, dt.month

    if month != prev_month:
        prev_month = month

        yy = year % 100
        fname = f"odiac2024_1km_excl_intl_{yy:02d}{month:02d}.tif"
        fpath = os.path.join(tif_root, str(year), fname)

        if (year == 2024 or year == 2025) and not os.path.exists(fpath):
            f_24 = os.path.join(tif_root, "2024",
                                f"odiac2024_1km_excl_intl_{24:02d}{month:02d}.tif")
            if os.path.exists(f_24):
                fpath = f_24
            else:
                f_23 = os.path.join(tif_root, "2023",
                                    f"odiac2024_1km_excl_intl_{23:02d}{month:02d}.tif")
                if os.path.exists(f_23):
                    fpath = f_23

        da = rxr.open_rasterio(fpath, masked=True).squeeze("band", drop=True)

        da = da.rename({"x": "lon", "y": "lat"})
        da = da.assign_coords(
            lon=((da.lon + 180) % 360) - 180
        ).sortby("lon")
        if da.lat.values[0] > da.lat.values[-1]:
            da = da.reindex(lat=da.lat[::-1])

        monthly_interp = da.interp(lat=lat, lon=lon, method="linear")
        tqdm.write(f"已插值 {year}-{month:02d} from {os.path.basename(fpath)}")

    data[it, :, :] = monthly_interp.values

ds = ds.assign(odiac=(("time", "lat", "lon"), data))
ds["odiac"].attrs.update({
    "long_name": "ODiAC CO2 excl intl",
    "units": "kg C per grid-cell per day"
})
ds.to_netcdf(output_nc)
